In [19]:
import pandas as pd
df = pd.read_csv('./nifty_ohh/NIFTY 50 - Minute data 2015 to Aug 2024.csv')
df.index = pd.to_datetime(df['date'])
# df.index.name = df.index.name.lower()
df.columns = df.columns.str.lower()

df['year'] = df.index.year.astype(int)
df['month'] = df.index.month.astype(int)
df['day'] = df.index.day.astype(int)
df['hour'] = df.index.hour.astype(int)
df['minute'] = df.index.minute.astype(int)
df['day_name'] = df.index.day_name()
df['max'] = df[['open', 'high', 'low', 'close']].max(axis=1)
df['min'] = df[['open', 'high', 'low', 'close']].min(axis=1)
df['difference'] = df['max'] - df['min']
df['candle_body'] = df['open'] - df['close']
df['high_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
df['low_shadow'] = df['low'] - df[['open', 'close']].min(axis=1)
df['low-open'] = df['low'] - df['open']
df['high-open'] = df['high'] - df['open']
df['trend'] = df['candle_body'].apply(lambda x: "DOWN" if x < 0 else "ZERO" if x == 0 else "UP")
main_df = df.sort_values(by=['year', 'month', 'day'], ascending=[True, True, True])
df_copy = main_df.copy()

In [33]:
df_copy = main_df.copy()
df = df_copy

In [ ]:
dfs = [group for _, group in df.groupby(['year','month', 'day'])]

df_1d = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'year': [df.iloc[0].year],
    'month': [df.iloc[0].month],
    'day': [df.iloc[0].day],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_1d = pd.concat([df_1d, ohlc_data])

In [21]:
# TO GENERATE 30 MINUTES DATAFRAME
# DATA FRAMES FROM 9:15 to 9:45

dfs = [group.head(30) for _, group in df.groupby(['year','month', 'day'])]

df_30m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'year': [df.iloc[0].year],
    'month': [df.iloc[0].month],
    'day': [df.iloc[0].day],
    'hour': [df.iloc[0].hour],
    'minute': [df.iloc[0].minute],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_30m = pd.concat([df_30m, ohlc_data])

In [26]:
# TO GENERATE 30 MINUTES DATAFRAME
# DATA FRAMES FROM 9:15 to 9:30

dfs = [group.head(15) for _, group in df.groupby(['year','month', 'day'])]

df_15m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'year': [df.iloc[0].year],
    'month': [df.iloc[0].month],
    'day': [df.iloc[0].day],
    'hour': [df.iloc[0].hour],
    'minute': [df.iloc[0].minute],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_15m = pd.concat([df_15m, ohlc_data])

In [34]:
# TO GENERATE 30 MINUTES DATAFRAME
# DATA FRAMES FROM 9:15 to 9:20

dfs = [group.head(5) for _, group in df.groupby(['year','month', 'day'])]

df_5m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'year': [df.iloc[0].year],
    'month': [df.iloc[0].month],
    'day': [df.iloc[0].day],
    'hour': [df.iloc[0].hour],
    'minute': [df.iloc[0].minute],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_5m = pd.concat([df_5m, ohlc_data])

In [35]:
df_5m.to_csv('NIFTY 50 - 5m data 2015 to Aug 2024.csv',index=True)

In [17]:
# TO GENERATE 30 MINUTES DATAFRAME
# DATA FRAMES FROM 9:15 to 9:20, 9:20 to 9:25
dfs = []

for _, group in df.groupby(['year','month', 'day']):
    group = group.head(10)
    for i in range(0, len(group), 5):
        dfs.append(group.iloc[i:i+5])

df_5m_5m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'year': [df.iloc[0].year],
    'month': [df.iloc[0].month],
    'day': [df.iloc[0].day],
    'hour': [df.iloc[0].hour],
    'minute': [df.iloc[0].minute],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_5m_5m = pd.concat([df_5m_5m, ohlc_data])

In [18]:
df_5m_5m

,high,low,open,close,year,month,day,hour,minute
2015-01-09,8301.30,8285.45,8285.45,8301.20,2015,1,9,9,15
2015-01-09,8303.00,8293.25,8300.50,8301.00,2015,1,9,9,20
2015-01-12,8291.35,8245.60,8291.35,8267.60,2015,1,12,9,15
2015-01-12,8274.45,8257.95,8267.45,8257.95,2015,1,12,9,20
2015-01-13,8355.90,8338.50,8346.15,8340.80,2015,1,13,9,15
...,...,...,...,...,...,...,...,...,...
2024-08-26,24907.70,24878.55,24886.45,24905.30,2024,8,26,9,20
2024-08-27,25037.70,24988.00,25024.80,25021.45,2024,8,27,9,15
2024-08-27,25025.90,25006.80,25023.30,25017.65,2024,8,27,9,20
2024-08-28,25048.15,25000.35,25030.80,25041.45,2024,8,28,9,15
